In [2]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

In [3]:
sns.set_theme(style="whitegrid", font_scale=3.)

### Val vs Test accuracy plot (Fig. 3)

In [4]:
# Load saved data
percent_acc = pd.read_csv('outputs/oracle_accuracy.csv')

In [5]:
f, ax = plt.subplots(figsize=(9, 9))
g = sns.scatterplot(data=percent_acc, x='validation', y='test', color='#0000FF', ax=ax, marker='+')
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Validation accuracy %', ylabel='Test accuracy %', xlim=[0, 100], ylim=[0, 100])
f.savefig('figures/fig3a.pdf', format='pdf', bbox_inches='tight')
plt.close(f)
print('saved')

saved


In [6]:
high_acc = percent_acc[percent_acc['validation'] >= 93.5].reset_index(drop=True)

f, ax = plt.subplots(figsize=(9, 9))
g = sns.scatterplot(data=high_acc, x='validation', y='test', color='#0000FF', ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Validation accuracy %', ylabel='Test accuracy %', xlim=[93.5, 95.5], ylim=[93.5, 95.5])
f.savefig('figures/fig3b.pdf', format='pdf', bbox_inches='tight')
plt.close(f)
print('saved')

saved


### Performance Comparison Plot (Fig. 4)

In [7]:
sns.set_theme(style="darkgrid", font_scale=1.25)

In [8]:
# number of samples vs validation accuracy
random_val_avg_by_samples = np.load('outputs/random_val_avg_by_samples.npy')
random_val_std_by_samples = np.load('outputs/random_val_std_by_samples.npy')

np_val_avg_by_samples = np.load('outputs/np_val_avg_by_samples.npy')
np_val_std_by_samples = np.load('outputs/np_val_std_by_samples.npy')

np2_val_avg_by_samples = np.load('outputs/np2_val_avg_by_samples.npy')
np2_val_std_by_samples = np.load('outputs/np2_val_std_by_samples.npy')

oracle_val_acc = np.load('outputs/oracle_val_acc.npy')
oracle_val_std = np.load('outputs/oracle_val_std.npy')

In [9]:
np_index = [i + 173 for i in range(len(np_val_avg_by_samples))]

In [10]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 500], ylim = [86, 96], xlabel='The number of samples', ylabel='Validation accuracy %',)

# neural predictor plot
ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')

plt.plot(np_index, np2_val_avg_by_samples, color='green', label='neural predictor w classifier')
plt.fill_between(range(173, 501), y1=np2_val_avg_by_samples-np2_val_std_by_samples, y2=np2_val_avg_by_samples+np2_val_std_by_samples, facecolor='green', alpha=0.2)

plt.plot(np_index, np_val_avg_by_samples, color='green', label='neural predictor w/o classifier', linestyle='dashed')
plt.fill_between(range(173, 501), y1=np_val_avg_by_samples-np_val_std_by_samples, y2=np_val_avg_by_samples+np_val_std_by_samples, facecolor='green', alpha=0.2)


# random search plot
plt.plot(random_val_avg_by_samples, color='red', label='random search', linestyle='dashed')
plt.fill_between(range(1, 501), y1=random_val_avg_by_samples-random_val_std_by_samples, y2=random_val_avg_by_samples+random_val_std_by_samples, facecolor='red', alpha=0.2)

# oracle line
ax.hlines(y=oracle_val_acc, xmin=0, xmax=500, color='black', label='oracle')
ax.fill_between(range(1, 501), y1=oracle_val_acc-oracle_val_std, y2=oracle_val_acc+oracle_val_std, facecolor='black', alpha=0.2)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/fig4a.pdf', format='pdf', bbox_inches='tight')

In [11]:
# number of samples vs test accuracy
random_test_avg_by_samples = np.load('outputs/random_test_avg_by_samples.npy')
random_test_std_by_samples = np.load('outputs/random_test_std_by_samples.npy')

np_test_avg_by_samples = np.load('outputs/np_test_avg_by_samples.npy')
np_test_std_by_samples = np.load('outputs/np_test_std_by_samples.npy')

np2_test_avg_by_samples = np.load('outputs/np2_test_avg_by_samples.npy')
np2_test_std_by_samples = np.load('outputs/np2_test_std_by_samples.npy')

oracle_test_acc = np.load('outputs/oracle_test_acc.npy')
oracle_test_std = np.load('outputs/oracle_test_std.npy')

In [12]:
np_index = [i + 173 for i in range(len(np_test_avg_by_samples))]

In [13]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 500], ylim = [85, 95], xlabel='The number of samples', ylabel='Test accuracy %',)

# neural predictor plot
ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')

plt.plot(np_index, np2_test_avg_by_samples, color='green', label='neural predictor w classifier')
plt.fill_between(range(173, 501), y1=np2_test_avg_by_samples-np2_test_std_by_samples, y2=np2_test_avg_by_samples+np2_test_std_by_samples, facecolor='green', alpha=0.2)

plt.plot(np_index, np_test_avg_by_samples, color='green', label='neural predictor w/o classifier', linestyle='dashed')
plt.fill_between(range(173, 501), y1=np_test_avg_by_samples-np_test_std_by_samples, y2=np_test_avg_by_samples+np_test_std_by_samples, facecolor='green', alpha=0.2)


# random search plot
plt.plot(random_test_avg_by_samples, color='red', label='random search', linestyle='dashed')
plt.fill_between(range(1, 501), y1=random_test_avg_by_samples-random_test_std_by_samples, y2=random_test_avg_by_samples+random_test_std_by_samples, facecolor='red', alpha=0.2)

# oracle line
ax.hlines(y=oracle_test_acc, xmin=0, xmax=500, color='black', label='oracle')
ax.fill_between(range(1, 501), y1=oracle_test_acc-oracle_test_std, y2=oracle_test_acc+oracle_test_std, facecolor='black', alpha=0.2)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/fig4b.pdf', format='pdf', bbox_inches='tight')

In [14]:
# number of samples vs test accuracy
random_test_avg = np.load('outputs/random_test_avg_by_time.npy')
random_test_std = np.load('outputs/random_test_std_by_time.npy')

np_test_avg = np.load('outputs/np_test_avg_by_time.npy')
np_test_std = np.load('outputs/np_test_std_by_time.npy')

np2_test_avg = np.load('outputs/np2_test_avg_by_time.npy')
np2_test_std = np.load('outputs/np2_test_std_by_time.npy')

oracle_test_acc = np.load('outputs/oracle_test_acc.npy')
oracle_test_std = np.load('outputs/oracle_test_std.npy')

In [15]:
np_index = [i + 134 for i in range(len(np_test_avg))]
np2_index = [i + 130 for i in range(len(np2_test_avg))]

In [16]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 500], ylim = [90., 95.], xlabel='Total training time x 1600 (seconds)', ylabel='Test accuracy %')

# neural predictor plot
ax.vlines(x=131, ymin=0, ymax=100, colors='green', linestyle='dotted')

plt.plot(np2_index, np2_test_avg, color='green', label='neural predictor w classifier')
plt.fill_between(range(131, 501), y1=np2_test_avg-np2_test_std, y2=np2_test_avg+np2_test_std, facecolor='green', alpha=0.2)

plt.plot(np_index, np_test_avg, color='green', label='neural predictor w/o classifier', linestyle='dashed')
plt.fill_between(range(134, 501), y1=np_test_avg-np_test_std, y2=np_test_avg+np_test_std, facecolor='green', alpha=0.2)


# random search plot
plt.plot(random_test_avg, color='red', label='random search', linestyle='dashed')
plt.fill_between(range(1, 501), y1=random_test_avg-random_test_std, y2=random_test_avg+random_test_std, facecolor='red', alpha=0.2)

# oracle line
ax.hlines(y=oracle_test_acc, xmin=0, xmax=500, color='black', label='oracle')
ax.fill_between(range(1, 501), y1=oracle_test_acc-oracle_test_std, y2=oracle_test_acc+oracle_test_std, facecolor='black', alpha=0.2)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/fig4c.pdf', format='pdf', bbox_inches='tight')

### Classifier Filtering (Fig. 6)

In [17]:
# load saved results
train_clf_val_acc = np.load('outputs/train_clf_data_val_acc.npy')

In [18]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 100], ylim=[0, 60], xlabel='Real validation accuracy %', ylabel='Counts')

plt.hist(train_clf_val_acc, color='red', bins=10)
plt.close()

f.savefig('figures/fig6a.pdf', format='pdf', bbox_inches='tight')

In [21]:
acc_set = np.load('outputs/clf_acc_set.npy')
inacc_set = np.load('outputs/clf_inacc_set.npy')

In [22]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [86, 95], ylim=[0, 14000], xlabel='Real validation accuracy %', ylabel='Counts')

plt.hist(acc_set, color='green', histtype='step', bins=500, label='accurate', linewidth=2)
plt.hist(inacc_set, color='red', histtype='step', bins=500, label='inaccurate', linewidth=2)

# show plot with legend
plt.legend(loc='upper left')
plt.close()

f.savefig('figures/fig6b.pdf', format='pdf', bbox_inches='tight')

### Trade-off N vs K (Fig. 7)

In [23]:
oracle_val = np.load('outputs/oracle_val_acc.npy')
oracle_test = np.load('outputs/oracle_test_acc.npy')

In [24]:
random_val = np.load('outputs/random_val_avg_2000.npy')
random_test = np.load('outputs/random_test_avg_2000.npy')

In [25]:
np_samples = np.load('outputs/ablation_study_by_samples.npy', allow_pickle=True).ravel()[0]
np_time = np.load('outputs/ablation_study_by_time.npy', allow_pickle=True).ravel()[0]

In [27]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 2000], ylim = [93, 95.4], xlabel='The number of samples', ylabel='Validation accuracy %',)

# neural predictor plot
ax.vlines(x=860, ymin=0, ymax=100, colors='pink', linestyle='dotted')
plt.plot([i + 860 for i in range(len(np_samples[860]['np_val_avg']))], np_samples[860]['np_val_avg'], color='pink', label='neural predictor 860', linewidth=3)

ax.vlines(x=344, ymin=0, ymax=100, colors='brown', linestyle='dotted')
plt.plot([i + 344 for i in range(len(np_samples[344]['np_val_avg']))], np_samples[344]['np_val_avg'], color='brown', label='neural predictor 344', linewidth=3)

ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')
plt.plot([i + 172 for i in range(len(np_samples[172]['np_val_avg']))], np_samples[172]['np_val_avg'], color='green', label='neural predictor 172', linewidth=3)

ax.vlines(x=129, ymin=0, ymax=100, colors='orange', linestyle='dotted')
plt.plot([i + 129 for i in range(len(np_samples[129]['np_val_avg']))], np_samples[129]['np_val_avg'], color='orange', label='neural predictor 129', linewidth=3)

ax.vlines(x=86, ymin=0, ymax=100, colors='purple', linestyle='dotted')
plt.plot([i + 86 for i in range(len(np_samples[86]['np_val_avg']))], np_samples[86]['np_val_avg'], color='purple', label='neural predictor 86', linewidth=3)

ax.vlines(x=43, ymin=0, ymax=100, colors='yellow', linestyle='dotted')
plt.plot([i + 43 for i in range(len(np_samples[43]['np_val_avg']))], np_samples[43]['np_val_avg'], color='yellow', label='neural predictor 43', linewidth=3)


# random search plot
plt.plot(random_val, color='red', label='random search', linestyle='dashed', linewidth=3)

# oracle line
ax.hlines(y=oracle_val, xmin=0, xmax=2000, color='black', label='oracle', linewidth=3)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/fig7a.pdf', format='pdf', bbox_inches='tight')

In [29]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 2000], ylim = [92.75, 94.75], xlabel='The number of samples', ylabel='Test accuracy %',)

# neural predictor plot
ax.vlines(x=860, ymin=0, ymax=100, colors='pink', linestyle='dotted')
plt.plot([i + 860 for i in range(len(np_samples[860]['np_test_avg']))], np_samples[860]['np_test_avg'], color='pink', label='neural predictor 860', linewidth=3)

ax.vlines(x=344, ymin=0, ymax=100, colors='brown', linestyle='dotted')
plt.plot([i + 344 for i in range(len(np_samples[344]['np_test_avg']))], np_samples[344]['np_test_avg'], color='brown', label='neural predictor 344', linewidth=3)

ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')
plt.plot([i + 172 for i in range(len(np_samples[172]['np_test_avg']))], np_samples[172]['np_test_avg'], color='green', label='neural predictor 172', linewidth=3)

ax.vlines(x=129, ymin=0, ymax=100, colors='orange', linestyle='dotted')
plt.plot([i + 129 for i in range(len(np_samples[129]['np_test_avg']))], np_samples[129]['np_test_avg'], color='orange', label='neural predictor 129', linewidth=3)

ax.vlines(x=86, ymin=0, ymax=100, colors='purple', linestyle='dotted')
plt.plot([i + 86 for i in range(len(np_samples[86]['np_test_avg']))], np_samples[86]['np_test_avg'], color='purple', label='neural predictor 86', linewidth=3)

ax.vlines(x=43, ymin=0, ymax=100, colors='yellow', linestyle='dotted')
plt.plot([i + 43 for i in range(len(np_samples[43]['np_test_avg']))], np_samples[43]['np_test_avg'], color='yellow', label='neural predictor 43', linewidth=3)


# random search plot
plt.plot(random_test, color='red', label='random search', linestyle='dashed', linewidth=3)

# oracle line
ax.hlines(y=oracle_test, xmin=0, xmax=2000, color='black', label='oracle', linewidth=3)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/fig7b.pdf', format='pdf', bbox_inches='tight')

In [30]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 500], ylim = [93, 94.75], xlabel='Total training time x 1600 (seconds)', ylabel='Test accuracy %')

# neural predictor plot
# ax.vlines(x=860, ymin=0, ymax=100, colors='pink', linestyle='dotted')
# plt.plot([i + 860 for i in range(len(np_samples[860]['np_test_avg']))], np_samples[860]['np_test_avg'], color='pink', label='neural predictor 860', linewidth=3)

ax.vlines(x=344, ymin=0, ymax=100, colors='brown', linestyle='dotted')
plt.plot([i + 344 for i in range(len(np_time[344]['np_test_avg']))], np_time[344]['np_test_avg'], color='brown', label='neural predictor 344', linewidth=3)

ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')
plt.plot([i + 172 for i in range(len(np_time[172]['np_test_avg']))], np_time[172]['np_test_avg'], color='green', label='neural predictor 172', linewidth=3)

ax.vlines(x=129, ymin=0, ymax=100, colors='orange', linestyle='dotted')
plt.plot([i + 129 for i in range(len(np_time[129]['np_test_avg']))], np_time[129]['np_test_avg'], color='orange', label='neural predictor 129', linewidth=3)

ax.vlines(x=86, ymin=0, ymax=100, colors='purple', linestyle='dotted')
plt.plot([i + 86 for i in range(len(np_time[86]['np_test_avg']))], np_time[86]['np_test_avg'], color='purple', label='neural predictor 86', linewidth=3)

ax.vlines(x=43, ymin=0, ymax=100, colors='yellow', linestyle='dotted')
plt.plot([i + 43 for i in range(len(np_time[43]['np_test_avg']))], np_time[43]['np_test_avg'], color='yellow', label='neural predictor 43', linewidth=3)


# random search plot
plt.plot(random_test, color='red', label='random search', linestyle='dashed', linewidth=3)

# oracle line
ax.hlines(y=oracle_test, xmin=0, xmax=2000, color='black', label='oracle', linewidth=3)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/fig7c.pdf', format='pdf', bbox_inches='tight')

### Ablation Study of Neural Predictor under different architectures (Appendix Fig. 1)

In [31]:
# number of samples vs validation accuracy
random_val = np.load('outputs/random_val_avg_2000.npy')
random_test = np.load('outputs/random_test_avg_2000.npy')

np_cnn = np.load('outputs/ablation_study_by_arch_cnn.npy', allow_pickle=True).ravel()[0]
np_mlp = np.load('outputs/ablation_study_by_arch_mlp.npy', allow_pickle=True).ravel()[0]
np_gcn = np.load('outputs/ablation_study_by_arch_gcn.npy', allow_pickle=True).ravel()[0]

oracle_val = np.load('outputs/oracle_val_acc.npy')
oracle_test = np.load('outputs/oracle_test_acc.npy')
oracle_val_std = np.load('outputs/oracle_val_std.npy')
oracle_test_std = np.load('outputs/oracle_test_std.npy')

In [32]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 2000], ylim = [92.5, 95.5], xlabel='The number of samples', ylabel='Validation accuracy %',)

# neural predictor plot
ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')
ax.vlines(x=172, ymin=0, ymax=100, colors='purple', linestyle='dotted')
ax.vlines(x=172, ymin=0, ymax=100, colors='blue', linestyle='dotted')

plt.plot([i + 172 for i in range(len(np_gcn['np_val_avg']))], np_gcn['np_val_avg'], color='green', label='GCN')
plt.fill_between(range(173, 2001), y1=np.array(np_gcn['np_val_avg'])-np.array(np_gcn['np_val_std']), y2=np.array(np_gcn['np_val_avg'])+np.array(np_gcn['np_val_std']), facecolor='green', alpha=0.2)

plt.plot([i + 172 for i in range(len(np_mlp['np_val_avg']))], np_mlp['np_val_avg'], color='purple', label='MLP')
plt.fill_between(range(173, 2001), y1=np.array(np_mlp['np_val_avg'])-np.array(np_mlp['np_val_std']), y2=np.array(np_mlp['np_val_avg'])+np.array(np_mlp['np_val_std']), facecolor='purple', alpha=0.2)

plt.plot([i + 172 for i in range(len(np_cnn['np_val_avg']))], np_cnn['np_val_avg'], color='blue', label='CNN')
plt.fill_between(range(173, 2001), y1=np.array(np_cnn['np_val_avg'])-np.array(np_cnn['np_val_std']), y2=np.array(np_cnn['np_val_avg'])+np.array(np_cnn['np_val_std']), facecolor='blue', alpha=0.2)


# random search plot
plt.plot(random_val, color='red', label='random search', linestyle='dashed')
# plt.fill_between(range(1, 501), y1=random_val-random_std, y2=random_val+random_std, facecolor='red', alpha=0.2)

# oracle line
ax.hlines(y=oracle_val, xmin=0, xmax=2000, color='black', label='oracle')
ax.fill_between(range(1, 2001), y1=oracle_val-oracle_val_std, y2=oracle_val+oracle_val_std, facecolor='black', alpha=0.2)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/figA1a.pdf', format='pdf', bbox_inches='tight')

In [33]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 2000], ylim = [92, 95], xlabel='The number of samples', ylabel='Test accuracy %',)

# neural predictor plot
ax.vlines(x=172, ymin=0, ymax=100, colors='green', linestyle='dotted')
ax.vlines(x=172, ymin=0, ymax=100, colors='purple', linestyle='dotted')
ax.vlines(x=172, ymin=0, ymax=100, colors='blue', linestyle='dotted')

plt.plot([i + 172 for i in range(len(np_gcn['np_test_avg']))], np_gcn['np_test_avg'], color='green', label='GCN')
plt.fill_between(range(173, 2001), y1=np.array(np_gcn['np_test_avg'])-np.array(np_gcn['np_test_std']), y2=np.array(np_gcn['np_test_avg'])+np.array(np_gcn['np_test_std']), facecolor='green', alpha=0.2)

plt.plot([i + 172 for i in range(len(np_mlp['np_test_avg']))], np_mlp['np_test_avg'], color='purple', label='MLP')
plt.fill_between(range(173, 2001), y1=np.array(np_mlp['np_test_avg'])-np.array(np_mlp['np_test_std']), y2=np.array(np_mlp['np_test_avg'])+np.array(np_mlp['np_test_std']), facecolor='purple', alpha=0.2)

plt.plot([i + 172 for i in range(len(np_cnn['np_test_avg']))], np_cnn['np_test_avg'], color='blue', label='CNN')
plt.fill_between(range(173, 2001), y1=np.array(np_cnn['np_test_avg'])-np.array(np_cnn['np_test_std']), y2=np.array(np_cnn['np_test_avg'])+np.array(np_cnn['np_test_std']), facecolor='blue', alpha=0.2)


# random search plot
plt.plot(random_test, color='red', label='random search', linestyle='dashed')
# plt.fill_between(range(1, 501), y1=random_test-random_std, y2=random_test+random_std, facecolor='red', alpha=0.2)

# oracle line
ax.hlines(y=oracle_test, xmin=0, xmax=2000, color='black', label='oracle')
ax.fill_between(range(1, 2001), y1=oracle_test-oracle_test_std, y2=oracle_test+oracle_test_std, facecolor='black', alpha=0.2)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/figA1b.pdf', format='pdf', bbox_inches='tight')

In [34]:
np_cnn = np.load('outputs/ablation_study_by_arch_time_cnn.npy', allow_pickle=True).ravel()[0]
np_mlp = np.load('outputs/ablation_study_by_arch_time_mlp.npy', allow_pickle=True).ravel()[0]
np_gcn = np.load('outputs/ablation_study_by_arch_time_gcn.npy', allow_pickle=True).ravel()[0]

In [35]:
f, ax = plt.subplots(figsize=(9, 9))

# axes settings
ax.set(xlim = [0, 500], ylim = [93.2, 94.75], xlabel='Total training time x 1600 (seconds)', ylabel='Test accuracy %')

# neural predictor plot
ax.vlines(x=500 - len(np_gcn['np_test_avg']), ymin=0, ymax=100, colors='green', linestyle='dotted')
ax.vlines(x=500 - len(np_mlp['np_test_avg']), ymin=0, ymax=100, colors='purple', linestyle='dotted')
ax.vlines(x=500 - len(np_cnn['np_test_avg']), ymin=0, ymax=100, colors='blue', linestyle='dotted')

plt.plot([i + 500 - len(np_gcn['np_test_avg']) for i in range(len(np_gcn['np_test_avg']))], np_gcn['np_test_avg'], color='green', label='GCN')
plt.fill_between(range(500 - len(np_gcn['np_test_avg']), 500), y1=np.array(np_gcn['np_test_avg'])-np.array(np_gcn['np_test_std']), y2=np.array(np_gcn['np_test_avg'])+np.array(np_gcn['np_test_std']), facecolor='green', alpha=0.2)

plt.plot([i + 500 - len(np_mlp['np_test_avg']) for i in range(len(np_mlp['np_test_avg']))], np_mlp['np_test_avg'], color='purple', label='MLP')
plt.fill_between(range(500 - len(np_mlp['np_test_avg']), 500), y1=np.array(np_mlp['np_test_avg'])-np.array(np_mlp['np_test_std']), y2=np.array(np_mlp['np_test_avg'])+np.array(np_mlp['np_test_std']), facecolor='purple', alpha=0.2)

plt.plot([i + 500 - len(np_cnn['np_test_avg']) for i in range(len(np_cnn['np_test_avg']))], np_cnn['np_test_avg'], color='blue', label='CNN')
plt.fill_between(range(500 - len(np_cnn['np_test_avg']), 500), y1=np.array(np_cnn['np_test_avg'])-np.array(np_cnn['np_test_std']), y2=np.array(np_cnn['np_test_avg'])+np.array(np_cnn['np_test_std']), facecolor='blue', alpha=0.2)


# random search plot
plt.plot(random_test, color='red', label='random search', linestyle='dashed')
# plt.fill_between(range(1, 501), y1=random_test-random_std, y2=random_test+random_std, facecolor='red', alpha=0.2)

# oracle line
ax.hlines(y=oracle_test, xmin=0, xmax=500, color='black', label='oracle')
ax.fill_between(range(1, 501), y1=oracle_test-oracle_test_std, y2=oracle_test+oracle_test_std, facecolor='black', alpha=0.2)

# show plot with legend
plt.legend(loc='lower right')
plt.close()

f.savefig('figures/figA1c.pdf', format='pdf', bbox_inches='tight')

### Performance of Neural Predictor (Fig. 9) -- NAS-Bench-101

In [36]:
sns.set_theme(style="whitegrid")

In [37]:
y_true = np.load('outputs/nasbench_y_true_train.npy')
y_pred = np.load('outputs/nasbench_y_pred_train.npy')

In [40]:
f, ax = plt.subplots(figsize=(5, 5))
g = sns.scatterplot(x=y_pred.ravel(), y=y_true, color='red', ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Predicted Model Validation Accuracy %', ylabel='Observed Model Validation Accuracy %', xlim=[80, 95.5], ylim=[80, 95.5], title=f'Training samples N={len(y_pred)}')
f.savefig('figures/fig9a-nasbench.pdf', format='pdf', bbox_inches='tight')
plt.close()

In [41]:
y_true = np.load('outputs/nasbench_y_true_val.npy')
y_pred = np.load('outputs/nasbench_y_pred_val.npy')

In [42]:
f, ax = plt.subplots(figsize=(5, 5))
g = sns.scatterplot(x=y_pred.ravel(), y=y_true, ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Predicted Model Validation Accuracy %', ylabel='Observed Model Validation Accuracy %', xlim=[80, 95.5], ylim=[80, 95.5], title=f'Validation samples N={len(y_pred)}')
f.savefig('figures/fig9b-nasbench.pdf', format='pdf', bbox_inches='tight')
plt.close()

In [43]:
y_true = np.load('outputs/nasbench_y_true_test_10.npy')
y_pred = np.load('outputs/nasbench_y_pred_test_10.npy')

In [45]:
f, ax = plt.subplots(figsize=(5, 5))
g = sns.scatterplot(x=y_pred, y=y_true, color='green', ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Predicted Model Validation Accuracy %', ylabel='Observed Model Validation Accuracy %', xlim=[82.5, 100], ylim=[82.5, 100], title=f'Test samples N={len(y_pred)}')
f.savefig('figures/fig9c-nasbench.pdf', format='pdf', bbox_inches='tight')
plt.close()

### Two-stage Neural Predictor

In [3]:
y_true = np.load('outputs/nasbench_two_y_true_train.npy')
y_pred = np.load('outputs/nasbench_two_y_pred_train.npy')

In [4]:
f, ax = plt.subplots(figsize=(5, 5))
g = sns.scatterplot(x=y_pred, y=y_true, color='red', ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Predicted Model Validation Accuracy %', ylabel='Observed Model Validation Accuracy %', xlim=[91, 95.5], ylim=[91, 95.5], title=f'Training samples N={len(y_pred)}')
f.savefig('figures/fig9a-nashbench-two.pdf', format='pdf', bbox_inches='tight')
plt.close()

In [5]:
y_true = np.load('outputs/nasbench_two_y_true_val.npy')
y_pred = np.load('outputs/nasbench_two_y_pred_val.npy')

In [6]:
f, ax = plt.subplots(figsize=(5, 5))
g = sns.scatterplot(x=y_pred, y=y_true, ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Predicted Model Validation Accuracy %', ylabel='Observed Model Validation Accuracy %', xlim=[91, 95.5], ylim=[91, 95.5], title=f'Validation samples N={len(y_pred)}')
f.savefig('figures/fig9b-nashbench-two.pdf', format='pdf', bbox_inches='tight')
plt.close()

In [7]:
y_true = np.load('outputs/nasbench_two_y_true_test_10.npy')
y_pred = np.load('outputs/nasbench_two_y_pred_test_10.npy')

In [8]:
f, ax = plt.subplots(figsize=(5, 5))
g = sns.scatterplot(x=y_pred, y=y_true, color='green', ax=ax)
g.plot([0, 1], [0, 1], transform=ax.transAxes, color='lightgray')
g.set(xlabel='Predicted Model Validation Accuracy %', ylabel='Observed Model Validation Accuracy %', xlim=[89, 99], ylim=[89, 99], title=f'Test samples N={len(y_pred)}')
f.savefig('figures/fig9c-nashbench-two.pdf', format='pdf', bbox_inches='tight')
plt.close()